# LIBRERIAS

In [329]:
#Importamos las librerias necesarias para el proyecto
import pandas as pd
import numpy as np
import json 
import ast
from textblob import TextBlob

# EXTRACCION



Se extraen los datos de las bases de datos que fueron brindadas.

In [330]:
# Abre el archivo JSON y carga su contenido en una lista de diccionarios
revs = []

with open('Datasets/australian_user_reviews.json', encoding='MacRoman') as d_user:
    for line in d_user.readlines():
        revs.append(ast.literal_eval(line))
        
# Crea un DataFrame a partir de la lista de diccionarios
df_user_revs = pd.DataFrame(revs)

In [331]:
# Volvemos a repetir el proceso en las otras 2 bases de datos
# El archivo JSON y carga su contenido en una lista de diccionarios
revs = []

with open('Datasets/australian_users_items.json', encoding='MacRoman') as d_user:
    for line in d_user.readlines():
        revs.append(ast.literal_eval(line))
        
# Crea un DataFrame a partir de la lista de diccionarios
df_user_items = pd.DataFrame(revs)

In [332]:
# Crea un DataFrame a partir del archivo JSON, en este caso utilizamos otro procedimiento para obtener el DataFrame
df_steam_output = pd.read_json('Datasets/output_steam_games.json', lines= True)

# ANALISIS

Se realiza un analisis del contenido de cada base de datos para determinar la utilidad de los mismos para el resto del proceso

In [333]:
df_user_revs.info() #Esta base de datos contiene las reviews de los usuarios.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [334]:
df_user_items.info() #Esta Base de datos contiene los juegos que tiene cada usuario y su uso.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [335]:
df_steam_output.info() #Esta base de datos contiene los juegos de Steam, con sus datos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [336]:
df_user_revs.head()
#Detectamos datos anidados en la comlumna REVIEWS. Seran desanidados y utilizados para un Sentiment Analisys.


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [337]:
df_user_items.head() 
#Se detectaron datos anidados en la comlumna ITEMS, se dividira en 4 columnas (ITEM_ID, ITEM_NAME, PLAYTIME_FOREVER, PLAYTIME_2WEEKS). 
#La columna ITEMS_COUNT debe ser transformada a un tipo de dato numerico.
#La nueva columna ITEM_ID debe contener datos de tipo numerico.
#La nueva columna ITEM_NAME debe contener datos de tipo object.
#la nueva columna PLAYTIME_FOREVER debe contener datos de tipo numerico.
#la nueva columna PLAYTIME_2WEEKS debe contener datos de tipo numerico.


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [338]:
#Ver solo el contenido de la columna items completa
df_user_items['items']

0        [{'item_id': '10', 'item_name': 'Counter-Strik...
1        [{'item_id': '10', 'item_name': 'Counter-Strik...
2        [{'item_id': '1200', 'item_name': 'Red Orchest...
3        [{'item_id': '10', 'item_name': 'Counter-Strik...
4        [{'item_id': '300', 'item_name': 'Day of Defea...
                               ...                        
88305    [{'item_id': '413850', 'item_name': 'CS:GO Pla...
88306    [{'item_id': '11020', 'item_name': 'TrackMania...
88307                                                   []
88308    [{'item_id': '304930', 'item_name': 'Unturned'...
88309                                                   []
Name: items, Length: 88310, dtype: object

In [339]:
df_steam_output.head()
#Se detectaron datos nulos en la comlumna ID, por lo que se eliminaran, esos registros, durante el proceso de transformacion.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [340]:
df_steam_output.tail()
#Se modificara el nombre de la columna ID por ITEM_ID para especificar que es el ID de un juego.
#Se modifica el tipo de dato de la columna ID_GAME a numerico.
#Se modifica el tipo de dato de la columna RELEASE_DATE a fecha.
#Se modifica el tipo de dato de la columna PRICE a numerico.
#Se modifica el tipo de dato de la columna EARLY_ACCESS a numerico.
#Se modifica el valor 'Free to Play' de la columna PRICE por 0.00 y su tipo de dato a numerico.
#Se detectan datos anidados en los registros GENRES, TAGS y SPECS. Se deben analizar cada uno de los casos para determinar como se utilizan o si alguna se elimina.


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


Se realiza un analisis mas profundo de los valores nulos en cada base de datos para determinar la utilidad de los mismos para el resto del proceso

In [341]:
#Se controlan los datos nulos 
df_user_revs.isnull().sum() #No se detectaron datos nulos en la base de datos.

user_id     0
user_url    0
reviews     0
dtype: int64

In [342]:
df_user_items.isnull().sum() #No se detectaron datos nulos en la base de datos.

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

In [343]:
df_steam_output.isnull().sum() #Se detectaron valores nulos.

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [344]:
#Se calcula el % de nulos en la base de datos.
df_steam_output.isnull().sum()/len(df_steam_output)*100 

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

In [345]:
#Se revisan los datos nulos
df_steam_output[df_steam_output.isnull().any(axis=1)] 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120431,None,None,Kalen Chock Presents: 2017 Free Tutorial,Kalen Chock Presents: 2017 Free Tutorial,http://store.steampowered.com/app/777950/Kalen...,2018-01-03,"[Design & Illustration, Tutorial]",http://steamcommunity.com/app/777950/reviews/?...,None,Free,0.0,777950.0,None
120433,VersoVR,"[Casual, Indie, Simulation, Sports, Early Access]",Cricket Club,Cricket Club,http://store.steampowered.com/app/772180/Crick...,January 2018,"[Early Access, Indie, Casual, Simulation, Sports]",http://steamcommunity.com/app/772180/reviews/?...,[Single-player],None,1.0,772180.0,VersoVR
120435,Retro Army Limited,"[Action, Adventure, Indie]",The spy who shot me™,The spy who shot me™,http://store.steampowered.com/app/771810/The_s...,2018-10-01,"[Action, Adventure, Indie]",http://steamcommunity.com/app/771810/reviews/?...,"[Single-player, Captions available]",None,0.0,771810.0,Retro Army Limited
120438,Riviysky,"[Casual, Indie]",BAE 2,BAE 2,http://store.steampowered.com/app/769330/BAE_2/,2018-01-04,"[Indie, Casual]",http://steamcommunity.com/app/769330/reviews/?...,[Single-player],None,0.0,769330.0,Riviysky


In [346]:
#Se revisan los datos que no son nulos en cada columna.
df_steam_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Se eliminaran los registros con datos nulos en la columna ID, ya que al no poseer un registro no pueden ser considerados para el analisis.

# TRANSFORMACION

## DF_STEAM_OUTPUT: to data_steam_output

Modificaciones y transformaciones de columnas a utilizar en las consultas de la API.

In [347]:
df_steam_output.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [348]:
#A partir del primer analisis de la base de datos, se detectaron registros nulos.
#Al no poseer registro 'id', no podemos considerar al registro como dato util para el analisis.
df_steam_output.dropna(subset=['id'],inplace=True)

### to DATA_STEAM_OUTPUT: 

Analisis y modificaciones necesarias de las columnas APP_NAME y TITLE y PRICE en el dataframe 

In [349]:
#Total de registros de la columna 'app_name'.
df_steam_output['app_name'].count()

32132

In [350]:
#Total de registros de la columna 'title'.
df_steam_output['title'].count()

30084

In [351]:
#Se Contabilizan la cantidad de null en la columna 'title'
df_steam_output['title'].isnull().sum()

2049

In [352]:
#Se Contabilizan la cantidad de null en la columna 'app_name'.
df_steam_output['app_name'].isnull().sum()

1

In [353]:
#Se consulta el unico registro que tiene null en la columna 'app_name'
df_steam_output[df_steam_output['app_name'].isnull()] 
#No posee el nombre del titulo pero es posible localizar el dato, por que posee el 'id' del juego, por ende el registro sera considerado para el analisis.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160.0,None


In [354]:
#Se consulta el porcentaje de datos iguales entre las columnas 'title' y 'app_name', para determinar si se puede eliminar una de ellas.
(df_steam_output['app_name'] == df_steam_output['title']).mean() * 100

91.89618149565867

In [355]:
#Se consulta el porcentaje de datos nulos en la columna 'title'.
(df_steam_output['title'].isnull()).mean() * 100

6.376622164130333

In [356]:
#Cantidad de registros que tienen datos iguales en las columnas 'title' y 'app_name'.
(df_steam_output['app_name'] == df_steam_output['title']).sum()

29529

In [357]:
#Registros que NO son nulos y que NO son iguales en las columnas 'title' y 'app_name'.
df_steam_output[(df_steam_output['app_name'] != df_steam_output['title']) & (df_steam_output['title'].notnull()) & (df_steam_output['app_name'].notnull())]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88390,Telltale Games,"[Action, Adventure]",Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock,http://store.steampowered.com/app/8200/Sam__Ma...,2006-10-17,"[Point & Click, Comedy, Adventure, Detective, ...",http://steamcommunity.com/app/8200/reviews/?br...,[Single-player],19.99,0.0,8200.0,Telltale Games
88393,Telltale Games,"[Action, Adventure]",Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy,http://store.steampowered.com/app/8210/Sam__Ma...,2006-12-20,"[Adventure, Action]",http://steamcommunity.com/app/8210/reviews/?br...,[Single-player],19.99,0.0,8210.0,Telltale Games
88419,Electronic Arts,[Strategy],Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3,http://store.steampowered.com/app/17480/Comman...,2008-10-28,"[Strategy, RTS, Base Building, Multiplayer, Co...",http://steamcommunity.com/app/17480/reviews/?b...,[Single-player],19.99,0.0,17480.0,EA Los Angeles
88492,Ubisoft,[Strategy],Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate,http://store.steampowered.com/app/15380/Heroes...,2006-11-14,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",http://steamcommunity.com/app/15380/reviews/?b...,[Single-player],9.99,0.0,15380.0,Nival
88494,Ubisoft,[Strategy],Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East,http://store.steampowered.com/app/15370/Heroes...,2007-10-16,"[Strategy, Turn-Based Strategy, Turn-Based, Fa...",http://steamcommunity.com/app/15370/reviews/?b...,[Single-player],9.99,0.0,15370.0,Nival
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120181,Telltale Games,"[Action, Adventure]",Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0,http://store.steampowered.com/app/8240/Sam__Ma...,2007-03-29,"[Adventure, Action]",http://steamcommunity.com/app/8240/reviews/?br...,[Single-player],19.99,0.0,8240.0,Telltale Games
120182,Telltale Games,"[Action, Adventure]",Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!,http://store.steampowered.com/app/8230/Sam__Ma...,2007-02-22,"[Adventure, Action, Point & Click, Free to Pla...",http://steamcommunity.com/app/8230/reviews/?br...,[Single-player],19.99,0.0,8230.0,Telltale Games
120183,Telltale Games,"[Action, Adventure]",Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon,http://store.steampowered.com/app/8250/Sam__Ma...,2007-04-26,"[Adventure, Action]",http://steamcommunity.com/app/8250/reviews/?br...,[Single-player],19.99,0.0,8250.0,Telltale Games
120208,Strategy First,"[Action, Strategy]",Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm,http://store.steampowered.com/app/6250/Making_...,2007-03-13,"[Strategy, Action, Turn-Based Strategy, Grand ...",http://steamcommunity.com/app/6250/reviews/?br...,"[Single-player, Multi-player]",4.99,0.0,6250.0,Muzzy Lane


In [358]:
#Detectamos que la columna 'title' posee el caracter "&amp" en lugar de "&", por lo que se procede a reemplazarlo.
# Contabiliza la cantidad de veces que aparece "&amp" dentro de la columna TITLE
df_steam_output['title'].str.contains('&amp').sum() 
#De los 555 registros que no son nulos, 550 continen el caracter "&amp" en la columna TITLE

550

In [359]:
#Porcentaje de registros que contienen el caracter "&amp" en la columna TITLE
(df_steam_output['title'].str.contains('&amp').sum()/df_steam_output['title'].count())*100

1.8282143332003724

In [360]:
#Se elimina la columna 'title' ya que:
    #Coincide en un 91,89 % con la columna 'app_name'.
    #El 6.37% de los datos son 'None'. 
    #El 1,82% son registros poseen algun error ortografico, de los cuales el 99,09% poseen el caracter '&amp' dentro del titulo.
df_steam_output.drop(columns=['title'],inplace=True) 


In [361]:
#Se modificara el nombre de la columna ID por ITEM_ID para especificar que es el ID de un juego.
df_steam_output.rename(columns={'id':'item_id'},inplace=True)

In [362]:
#Se modifica el valor 'Free To Play' y 'Free to Use' de la columna PRICE por 0.00.
df_steam_output['price'] = df_steam_output['price'].replace('Free To Play', '0.00') 
df_steam_output['price'] = df_steam_output['price'].replace('Free to Use', '0.00')
df_steam_output['price'] = df_steam_output['price'].replace('Free to Play', '0.00')


In [363]:
df_steam_output.head()

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


In [364]:
df_steam_output.head()

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


Transformaciones de los tipos de datos del dataframe.

In [365]:
data_steam_output = df_steam_output.copy() 

In [366]:
#Se divide la columna 'release_date' en 3 columnas, una con la mes y el dia.
data_steam_output[['release_year','release_month', 'release_day']] = data_steam_output['release_date'].str.split('-',expand=True)
#Se reorganizan las columnas del dataframe.
data_steam_output = data_steam_output[['item_id', 'app_name', 'developer', 'publisher', 'release_year', 'release_month', 'release_day', 'price', 'early_access', 'url', 'reviews_url']]


In [367]:
data_steam_output.head()

,item_id,app_name,developer,publisher,release_year,release_month,release_day,price,early_access,url,reviews_url
88310,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018,01,04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
88311,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018,01,04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017,07,24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
88313,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017,12,07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
88314,773570.0,Log Challenge,None,None,None,None,None,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


In [368]:
#Valores de release_year
data_steam_output['release_year'].value_counts()

release_year
2017                                                                                          9538
2016                                                                                          6934
2015                                                                                          4945
2014                                                                                          2832
2013                                                                                          1551
                                                                                              ... 
0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵       1
Fall 2018                                                                                        1
2018年1月                                                                                          1
预热群52756441                                                                                     

In [369]:
data_steam_output[data_steam_output['release_year'].str.isnumeric() == False]

,item_id,app_name,developer,publisher,release_year,release_month,release_day,price,early_access,url,reviews_url
88320,768570.0,Uncanny Islands,Qucheza,Qucheza,Soon..,None,None,None,1.0,http://store.steampowered.com/app/768570/Uncan...,http://steamcommunity.com/app/768570/reviews/?...
88354,714020.0,Idle Adventure,"zha7,Phaedruns,Yingke Wu",zha7,Beta测试已开启,None,None,None,1.0,http://store.steampowered.com/app/714020/Idle_...,http://steamcommunity.com/app/714020/reviews/?...
88560,29160.0,Blueberry Garden,Erik Svedäng,Erik Svedäng,Jun 2009,None,None,4.99,0.0,http://store.steampowered.com/app/29160/Bluebe...,http://steamcommunity.com/app/29160/reviews/?b...
88816,61510.0,Age of Wonders II: The Wizard's Throne,Triumph Studios,Paradox Interactive,Oct 2010,None,None,9.99,0.0,http://store.steampowered.com/app/61510/Age_of...,http://steamcommunity.com/app/61510/reviews/?b...
88819,61520.0,Age of Wonders Shadow Magic,Triumph Studios,Paradox Interactive,Oct 2010,None,None,9.99,0.0,http://store.steampowered.com/app/61520/Age_of...,http://steamcommunity.com/app/61520/reviews/?b...
...,...,...,...,...,...,...,...,...,...,...,...
120391,716780.0,Sand is the Soul,MGP Studios,Fat Dog Games,Q1 2018,None,None,None,0.0,http://store.steampowered.com/app/716780/Sand_...,http://steamcommunity.com/app/716780/reviews/?...
120392,716800.0,War Builder League,Cyber Prophet,Fat Dog Games,15.01.2018,None,None,None,0.0,http://store.steampowered.com/app/716800/War_B...,http://steamcommunity.com/app/716800/reviews/?...
120393,718250.0,Chromalocity,Wrecked Angle Studios,Wrecked Angle Studios,"Q1, 2018",None,None,None,1.0,http://store.steampowered.com/app/718250/Chrom...,http://steamcommunity.com/app/718250/reviews/?...
120397,755830.0,Lonely Astronaut,"Greyson Richey,Nicholas Lives","Greyson Richey, Nicholas Lives",Coming Soon,None,None,None,0.0,http://store.steampowered.com/app/755830/Lonel...,http://steamcommunity.com/app/755830/reviews/?...


In [370]:
data_steam_output['release_year'] = pd.to_numeric(data_steam_output['release_year'], errors='coerce')


In [371]:
#Valores de release_month
data_steam_output['release_month'].value_counts()

release_month
10             3207
11             2936
12             2914
09             2848
08             2707
07             2467
06             2295
05             2274
04             2171
03             2137
02             1953
01             1873
 Early 2017       1
Name: count, dtype: int64

In [372]:
#Se modifica el valor 'Early 2017' por 01 en la columna 'release_month'.
data_steam_output['release_month'] = data_steam_output['release_month'].replace(' Early 2017', '01')

In [373]:
#Valores de release_day
data_steam_output['release_day'].value_counts()

release_day
01    1423
15    1165
20    1124
16    1100
21    1079
26    1029
14    1025
19    1003
28     988
29     984
07     979
25     973
12     967
03     950
22     947
08     939
30     938
13     933
06     922
24     920
18     917
10     916
17     910
05     872
27     867
11     866
09     865
04     849
02     844
23     822
31     666
Name: count, dtype: int64

In [374]:
# Concatenar las columnas de fecha para formar 'release_date' en una sola operación
data_steam_output['release_date'] = data_steam_output['release_year'].astype(str) + '-' + data_steam_output['release_month'].astype(str) + '-' + data_steam_output['release_day'].astype(str)

# Luego, se eliminan las columnas 'release_year', 'release_month', y 'release_day'.
data_steam_output = data_steam_output.drop(['release_year', 'release_month', 'release_day'], axis=1)

#Se reorganizan las columnas del dataframe.
data_steam_output = data_steam_output[['item_id', 'app_name', 'developer', 'publisher', 'release_date', 'price', 'early_access', 'url', 'reviews_url']]

Version final: DATA_STEAM_OUTPUT

In [375]:
data_steam_output.head()

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
88310,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018.0-01-04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
88311,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018.0-01-04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017.0-07-24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
88313,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017.0-12-07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
88314,773570.0,Log Challenge,None,None,nan-None-None,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


## DF_USER_REVS: to data_users_revs

Modificaciones y transformaciones de columnas con datos anidados.

In [376]:
df_user_revs.head()
#Detectamos datos anidados en la comlumna REVIEWS. Seran desanidados y utilizados para un Sentiment Analysis


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [377]:
#Se revisa el contenido de un usuario al azar en la columna reviews, para determinar como se desanidara la columna.
df_user_revs['reviews'][3]

[{'funny': '',
  'posted': 'Posted October 14, 2013.',
  'last_edited': '',
  'item_id': '250320',
  'helpful': '2 of 2 people (100%) found this review helpful',
  'recommend': True,
  'review': 'This game... is so fun. The fight sequences have been improved from walking dead. It also includes more of a Sam and Max puzzle solving (some of it in the first episode) and walking dead. The game also gets even more better if you have read the Fables comic books, which are without a doubt, very good. The music is also superb and fit the scenarios very well.'},
 {'funny': '',
  'posted': 'Posted July 28, 2012.',
  'last_edited': '',
  'item_id': '20920',
  'helpful': '1 of 1 people (100%) found this review helpful',
  'recommend': True,
  'review': "Really Really Really Great Game, very good story, im in chapter 1 atm and i think its great. You get a really early link with characters. No need to play the first game, its nicely wrapped up for you in a five minute video. FYI beware of the sex sc

In [378]:
# Función para separar los datos en la columna 'reviews' con el nuevo formato
def separate_reviews(df):
    new_data = []  # Crea una lista vacía para almacenar los datos separados

    for _, row in df.iterrows():  
        user_id = row['user_id']

        for review in row['reviews']:  
            review_data = {  # Crea un diccionario para almacenar los datos separados
                'user_id': user_id,  
                'funny': review['funny'],  
                'posted': review['posted'],  
                'last_edited': review['last_edited'],  
                'item_id': review['item_id'],  
                'helpful': review['helpful'],  
                'recommend': review['recommend'],  
                'review': review['review']  
            }
            new_data.append(review_data)  
    
    return pd.DataFrame(new_data)  # Convierte la lista de datos separados en un nuevo DataFrame y lo retorna


### to DATA_USERS_REVS:

Se crea un nuevo dataframe con los datos necesarios pero desanidados en sus columnas correspondientes.

In [379]:
data_users_revs = separate_reviews(df_user_revs)

In [380]:
data_users_revs.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [381]:
#Se analiza el contenido de las columnas 'funny'.
data_users_revs['funny'].isnull().sum()/len(data_users_revs)*100 ### Revisar si con otro encoding se puede solucionar el problema de la columna 'funny'

0.0

In [382]:
#Se analiza el contenido de las columnas 'posted'.
data_users_revs['last_edited'].isnull().sum()/len(data_users_revs)*100

0.0

In [383]:
#Se analiza el contenido de las columnas 'last_edited'.
data_users_revs['helpful'].isnull().sum()/len(data_users_revs)*100

0.0

In [384]:
#Se revisa como esta compuesto el contenido de la columna 'funny'.
data_users_revs['funny'].value_counts()

funny
                                        51154
1 person found this review funny         5180
2 people found this review funny         1232
3 people found this review funny          491
4 people found this review funny          267
                                        ...  
58 people found this review funny           1
405 people found this review funny          1
105 people found this review funny          1
1,130 people found this review funny        1
825 people found this review funny          1
Name: count, Length: 186, dtype: int64

In [385]:
#Se revisa como esta compuesto el contenido de la columna 'posted'.
data_users_revs['last_edited'].value_counts()

last_edited
                                  53165
Last edited November 25, 2013.       99
Last edited October 17, 2015.        19
Last edited June 6, 2015.            18
Last edited January 3.               17
                                  ...  
Last edited May 30, 2015.             1
Last edited May 21, 2015.             1
Last edited February 11, 2014.        1
Last edited May 8, 2014.              1
Last edited August 15, 2014.          1
Name: count, Length: 1015, dtype: int64

In [386]:
#Se revisa como esta compuesto el contenido de la columna 'last_edited'.
data_users_revs['helpful'].value_counts()

helpful
No ratings yet                                     30168
1 of 1 people (100%) found this review helpful      6730
0 of 1 people (0%) found this review helpful        4024
1 of 2 people (50%) found this review helpful       2493
2 of 2 people (100%) found this review helpful      1872
                                                   ...  
73 of 96 people (76%) found this review helpful        1
12 of 31 people (39%) found this review helpful        1
12 of 53 people (23%) found this review helpful        1
17 of 37 people (46%) found this review helpful        1
24 of 34 people (71%) found this review helpful        1
Name: count, Length: 1344, dtype: int64

In [387]:
#Se eliminan las columnas FUNNY, LAST_EDITED, HELPFUL ya que no son necesarias para el analisis.
data_users_revs.drop(columns=['funny','last_edited','helpful'],inplace=True)

In [388]:
#En base al contenido de la columna 'posted' se hacen las siguientes modificaciones:
#Se eliminan los caracteres que no son necesarios
data_users_revs['posted'] = data_users_revs['posted'].str.replace('Posted', '')

#Se revisan los valores de la columna posted
data_users_revs['posted']

0          November 5, 2011.
1             July 15, 2011.
2            April 21, 2011.
3             June 24, 2014.
4         September 8, 2013.
                ...         
59300               July 10.
59301                July 8.
59302                July 3.
59303               July 20.
59304                July 2.
Name: posted, Length: 59305, dtype: object

In [389]:
#Se divide la columna 'posted' en 2 columnas, una con la mes y el dia y otra con el año
data_users_revs[['rev_date','rev_year']] = data_users_revs['posted'].str.split(',',expand=True)

In [390]:
#Se elimina la columna 'posted' ya que no es necesaria.
data_users_revs.drop(columns=['posted'],inplace=True)

In [391]:
data_users_revs.head()

,user_id,item_id,recommend,review,rev_date,rev_year
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,November 5,2011.
1,76561197970982479,22200,True,It's unique and worth a playthrough.,July 15,2011.
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,April 21,2011.
3,js41637,251610,True,I know what you think when you see this title ...,June 24,2014.
4,js41637,227300,True,For a simple (it's actually not all that simpl...,September 8,2013.


In [392]:
#Se elimina el "." del 'rev_year'.
data_users_revs['rev_year'] = data_users_revs['rev_year'].str.replace('.', '')

In [393]:
#Ver la cantidad de valores de la columna 'rev_date'.
data_users_revs['rev_date'].value_counts()


rev_date
 December 27      368
 December 23      365
 December 29      336
 December 26      333
 December 28      331
                 ... 
 September 6.      17
 August 10.        15
 September 14.     12
 September 13.     10
 February 29        2
Name: count, Length: 628, dtype: int64

In [394]:
#Eliminar el "." al final de los valores de la columna 'rev_date'
data_users_revs['rev_date'] = data_users_revs['rev_date'].str.replace('.', '')

In [395]:
# Se dividide 'rev_date' en 'rev_day' y 'rev_month', ya que son datos necesarios para las consultas del modelo a desarrollar.
data_users_revs['rev_day'] = data_users_revs['rev_date'].str.extract(r'(\d+)')  # Extraer el valor numérico del día
data_users_revs['rev_month'] = data_users_revs['rev_date'].str.extract(r'(\b\w+\b)')  # Extraer el nombre del mes
data_users_revs.drop(columns=['rev_date'],inplace=True)

In [396]:
#Pasar la columna review al final del dataframe
data_users_revs = data_users_revs[['user_id', 'item_id', 'recommend', 'rev_year', 'rev_month', 'rev_day', 'review']]

In [397]:
#Revisar valores nulos en las columnas 'rev_year', 'rev_month' y 'rev_day'
data_users_revs['rev_year'].isnull().sum() 

10119

In [398]:
#Porcentaje de valores nulos en la columna 'rev_year'
(data_users_revs['rev_year'].isnull()).mean() * 100 #El 17% de los datos son nulos, seran consideradas las fechas de lanzamiento de los juegos para esas reviews.

17.062642272995532

In [399]:
#Revisar valores nulos en las columnas 'rev_month'.
data_users_revs['rev_month'].isnull().sum() #No posee valores nulos


0

In [400]:
#Revisar valores nulos en las columnas 'rev_day'
data_users_revs['rev_day'].isnull().sum() #No posee valores nulos

0

In [401]:
data_users_revs.describe() #Se revisan los datos nuevamente para ver si se detecta alguna anomalia.

,user_id,item_id,recommend,rev_year,rev_month,rev_day,review
count,59305,59305,59305,49186,59305,59305,59305
unique,25458,3682,2,6,12,31,55313
top,76561198094665607,730,True,2014,December,28,good game
freq,20,3759,52473,22111,8014,2234,100


In [402]:
#Reordenamos las columnas del dataframe. pasar recomend al final.
data_users_revs = data_users_revs[['user_id', 'item_id', 'rev_year', 'rev_month', 'rev_day', 'review', 'recommend']]

Version final: DATA_USERS_REVS

In [403]:
data_users_revs.head()

,user_id,item_id,rev_year,rev_month,rev_day,review,recommend
0,76561197970982479,1250,2011,November,5,Simple yet with great replayability. In my opi...,True
1,76561197970982479,22200,2011,July,15,It's unique and worth a playthrough.,True
2,76561197970982479,43110,2011,April,21,Great atmosphere. The gunplay can be a bit chu...,True
3,js41637,251610,2014,June,24,I know what you think when you see this title ...,True
4,js41637,227300,2013,September,8,For a simple (it's actually not all that simpl...,True


## DF_USER_ITEMS:  to data_users_items

Modificaciones y transformaciones de columnas con datos anidados.

In [404]:
df_user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [405]:
#Se detectaron datos anidados en la comlumna ITEMS, se dividira en 4 columnas (ITEM_ID, ITEM_NAME, PLAYTIME_FOREVER, PLAYTIME_2WEEKS).

def separate_items(df):
    new_data = []  # Crea una lista vacía para almacenar los datos separados
    
    for _, row in df.iterrows():  
        user_id = row['user_id']
        items_count = row['items_count']
        steam_id = row['steam_id']        
        user_url = row['user_url']
        
        for item in row['items']:  
            item_data = {  # Crea un diccionario para almacenar los datos separados
                'user_id': user_id,
                'items_count': items_count,
                'steam_id': steam_id,
                'item_id': item['item_id'],  
                'item_name': item['item_name'],  
                'playtime_forever': item['playtime_forever'],  
                'playtime_2weeks': item['playtime_2weeks'],
                'user_url': user_url  
            }
            new_data.append(item_data)  
    
    return pd.DataFrame(new_data)  # Convierte la lista de datos separados en un nuevo DataFrame y lo retorna

### to DATA_USERS_ITEMS:

Se crea un nuevo dataframe con los datos necesarios pero desanidados en sus columnas correspondientes.

In [406]:
data_users_items = separate_items(df_user_items)

In [407]:
#Se cambia el orden de las columnas del dataframe.
data_users_items = data_users_items[['user_id', 'items_count', 'steam_id', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks', 'user_url']] 

In [408]:
data_users_items.head()

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


In [409]:
#Cantidad de registros unicos en la columna 'steam_id'
data_users_items['user_id'].nunique() 

70912

In [410]:
def assign_user_index_count(df):
    '''
    Este código recorre la columna steam_id, compara cada valor con el valor anterior 
    y aumenta user_index_count si el valor cambia. 
    Luego, agrega los valores de user_index_count a una lista 
    y finalmente agrega esa lista como una nueva columna en el DataFrame original.
    '''
    user_index_count = 0
    user_index_counts = []
    
    for i, steam_id in enumerate(df['steam_id']):
        if i == 0 or steam_id != df['steam_id'].iloc[i - 1]:
            user_index_count += 1
        user_index_counts.append(user_index_count)

    df['user_index_count'] = user_index_counts
    
    return df

In [411]:
data_users_items = assign_user_index_count(data_users_items)

In [412]:
data_users_items.head()

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,user_url,user_index_count
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...,1
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...,1
2,76561197970982479,277,76561197970982479,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...,1
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...,1
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...,1


In [413]:
#ver el user index count 23
data_users_items[data_users_items['user_index_count'] == 23]


,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks,user_url,user_index_count
5379,DJKamBer,152,76561198026039330,4000,Garry's Mod,868,0,http://steamcommunity.com/id/DJKamBer,23
5380,DJKamBer,152,76561198026039330,1250,Killing Floor,2851,0,http://steamcommunity.com/id/DJKamBer,23
5381,DJKamBer,152,76561198026039330,35420,Killing Floor Mod: Defence Alliance 2,2,0,http://steamcommunity.com/id/DJKamBer,23
5382,DJKamBer,152,76561198026039330,23490,Tropico 3 - Steam Special Edition,0,0,http://steamcommunity.com/id/DJKamBer,23
5383,DJKamBer,152,76561198026039330,42120,Lead and Gold - Gangs of the Wild West,0,0,http://steamcommunity.com/id/DJKamBer,23
...,...,...,...,...,...,...,...,...,...
5526,DJKamBer,152,76561198026039330,11130,Sherlock Holmes: The Mystery of The Mummy,0,0,http://steamcommunity.com/id/DJKamBer,23
5527,DJKamBer,152,76561198026039330,348840,UFHO2,0,0,http://steamcommunity.com/id/DJKamBer,23
5528,DJKamBer,152,76561198026039330,48000,LIMBO,0,0,http://steamcommunity.com/id/DJKamBer,23
5529,DJKamBer,152,76561198026039330,273350,Evolve Stage 2,0,0,http://steamcommunity.com/id/DJKamBer,23


In [414]:
#Se modifica el orden de las columnas.
data_users_items = data_users_items[['user_id', 'items_count', 'steam_id', 'user_index_count', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks', 'user_url']]

In [415]:
data_users_items.value_counts('user_index_count')


user_index_count
3231     7762
16618    6700
957      6410
5292     6132
5267     5027
         ... 
51142       1
65958       1
65957       1
51158       1
55174       1
Name: count, Length: 71339, dtype: int64

Version final: DATA_USERS_ITEMS

In [416]:
data_users_items.head() #Se agrego la columna 'user_index_count', mas adelante sera utilizada para acceder mas rapido a los datos.


,user_id,items_count,steam_id,user_index_count,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,1,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,1,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,1,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,1,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,1,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


## DF_STEAM_OUTPUT: to data_genre & to data_specs

Se vuelve a utilizar este dataframe para dividirlo en distintas bases de datos, para mejorar el acceso a la informacion en las futuras consultas de la API. Tambien se elimina una columna incesesaria para el resto del proyecto.

### to DATA_GENRE

Se trabaja en una base de datos solo para la columna de GENEROS

In [417]:
df_genre_only = df_steam_output.copy() #Se crea una copia del dataframe para trabajar con el.

In [418]:
df_genre_only.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24082 non-null  object 
 1   genres        28851 non-null  object 
 2   app_name      32132 non-null  object 
 3   url           32133 non-null  object 
 4   release_date  30067 non-null  object 
 5   tags          31971 non-null  object 
 6   reviews_url   32133 non-null  object 
 7   specs         31464 non-null  object 
 8   price         30756 non-null  object 
 9   early_access  32133 non-null  float64
 10  item_id       32133 non-null  float64
 11  developer     28835 non-null  object 
dtypes: float64(2), object(10)
memory usage: 3.2+ MB


In [419]:
#Se eliminan todas las columnas menos 'item_id' y 'genres'
df_genre_only.drop(columns=['publisher', 'url', 'release_date', 'tags', 'reviews_url', 'specs' ,'price', 'early_access', 'developer'],inplace=True)

In [420]:
df_genre_only.head()

,genres,app_name,item_id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,670290.0
88313,"[Action, Adventure, Casual]",弹炸人2222,767400.0
88314,None,Log Challenge,773570.0


In [421]:
#Existencia de Datos nulos
df_genre_only['genres'].isnull().sum() 

3282

In [422]:
#Ver datos nulos
df_genre_only[df_genre_only['genres'].isnull()] #Los datos nulos, seran considerados como "no data".

,genres,app_name,item_id
88314,None,Log Challenge,773570.0
88321,None,Icarus Six Sixty Six,724910.0
88329,None,After Life VR,772590.0
88330,None,Kitty Hawk,640250.0
88332,None,Mortars VR,711440.0
...,...,...,...
120428,None,Robotpencil Presents: Exercise: Brushwork,775640.0
120429,None,Robotpencil Presents: Creative Composition,777930.0
120430,None,The Gamble House,775370.0
120431,None,Kalen Chock Presents: 2017 Free Tutorial,777950.0


In [423]:
#Se reemplazan los datos nulos por "no data"
df_genre_only['genres'].fillna('No data', inplace=True) 

In [424]:
df_genre_only['genres'].value_counts()

genres
No data                                                           3282
[Action]                                                          1880
[Action, Indie]                                                   1650
[Simulation]                                                      1396
[Casual, Simulation]                                              1359
                                                                  ... 
[Action, Adventure, Racing, Simulation, Strategy]                    1
[Action, Adventure, Casual, Indie, Racing, Sports, Strategy]         1
[Action, Adventure, Casual, Indie, Racing, Simulation, Sports]       1
[Action, Massively Multiplayer, RPG, Strategy]                       1
[Adventure, Casual, RPG, Simulation, Early Access]                   1
Name: count, Length: 884, dtype: int64

In [425]:
df_genre_only.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genres    32133 non-null  object 
 1   app_name  32132 non-null  object 
 2   item_id   32133 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1004.2+ KB


In [426]:
df_genre_only.head()

,genres,app_name,item_id
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140.0
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,670290.0
88313,"[Action, Adventure, Casual]",弹炸人2222,767400.0
88314,No data,Log Challenge,773570.0


In [427]:
def separate_genres(df):
    #Obtiene la lista completa de generos presentes en el DataFrame
    all_genres = set() # Crea un conjunto para almacenar los géneros
    for genres in df['genres']:
        if isinstance(genres, list):
            all_genres.update(genres)
            
    # Crea columnas para cada genero encontrado
    for genre in all_genres:
        df[genre] = df['genres'].apply(lambda x: 1 if isinstance(x, list) and genre in x else 0)
        
    #Elimina la columna 'genres' original
    df = df.drop(columns=['genres'])

    return df

In [428]:
data_genre = separate_genres(df_genre_only)

Version final: DATA_GENRE

In [429]:
data_genre.head()

,app_name,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
88310,Lost Summoner Kitty,761140.0,0,0,0,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
88311,Ironbound,643980.0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,1,0,0
88312,Real Pool 3D - Poolians,670290.0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0
88313,弹炸人2222,767400.0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
88314,Log Challenge,773570.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [430]:
data_genre.info()


<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   app_name                   32132 non-null  object 
 1   item_id                    32133 non-null  float64
 2   Software Training          32133 non-null  int64  
 3   Animation &amp; Modeling   32133 non-null  int64  
 4   Utilities                  32133 non-null  int64  
 5   Simulation                 32133 non-null  int64  
 6   Strategy                   32133 non-null  int64  
 7   Indie                      32133 non-null  int64  
 8   Accounting                 32133 non-null  int64  
 9   Racing                     32133 non-null  int64  
 10  Design &amp; Illustration  32133 non-null  int64  
 11  Adventure                  32133 non-null  int64  
 12  Education                  32133 non-null  int64  
 13  Early Access               32133 non-null  int

In [431]:
data_genre.describe()
#Se presenta la existencia de datos repetidos en las columnas 'item_id' y 'app_name'.

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
count,3.213300e+04,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,...,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000
mean,4.517574e+05,0.003268,0.005695,0.010581,0.208477,0.216506,0.493511,0.000218,0.033704,0.014316,...,0.039119,0.257741,0.003610,0.002894,0.002396,0.008340,0.170510,0.063206,0.034482,0.352286
std,1.827140e+05,0.057071,0.075252,0.102320,0.406226,0.411870,0.499966,0.014758,0.180468,0.118790,...,0.193880,0.437398,0.059976,0.053721,0.048894,0.090945,0.376086,0.243337,0.182466,0.477690
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.292800e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.520600e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.934000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2.028850e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Se elimina la columna TAGs

No son necesarios los datos para el trabajo que se nos pide, por ende se elimina la columna correspondiente.

In [432]:
df_steam_output.head()

,publisher,genres,app_name,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None


In [433]:
df_tags_only = df_steam_output.copy() #Se crea una copia del dataframe para trabajar con el.

In [434]:
#Se revisa y analisa el contenido de la columna 'tags'.
df_tags_only['tags'].value_counts()

tags
[Casual, Simulation]                                                                                                             1292
[Action]                                                                                                                         1105
[Simulation]                                                                                                                     1085
[Strategy, RPG, Indie]                                                                                                            632
[Strategy]                                                                                                                        625
                                                                                                                                 ... 
[Simulation, Indie, Action, Adventure, Funny, Open World, First-Person, Sandbox, Free to Play]                                      1
[Indie, Casual, Puzzle, Difficult]                       

In [435]:
#Se elimina la columna 'tags', del dataframe original, ya que no tiene datos relevantes para el modelo.
df_steam_output.drop(columns=['tags'],inplace=True)

### to DATA_SPECS

Se trabaja en una base de datos solo para la columna de ESPECIFICACIONES.

In [436]:
df_specs_only = df_steam_output.copy() #Se crea una copia del dataframe para trabajar con el.
df_specs_only.drop(columns=['url', 'release_date', 'reviews_url', 'genres', 'price', 'early_access', 'developer', 'publisher'],inplace=True)
#Se reorganizan las columnas del dataframe.
df_specs_only = df_specs_only[['item_id', 'app_name', 'specs']]
df_specs_only.head()

,item_id,app_name,specs
88310,761140.0,Lost Summoner Kitty,[Single-player]
88311,643980.0,Ironbound,"[Single-player, Multi-player, Online Multi-Pla..."
88312,670290.0,Real Pool 3D - Poolians,"[Single-player, Multi-player, Online Multi-Pla..."
88313,767400.0,弹炸人2222,[Single-player]
88314,773570.0,Log Challenge,"[Single-player, Full controller support, HTC V..."


In [437]:
#Existencia de Datos nulos
df_specs_only['specs'].isnull().sum() 

669

In [438]:
#Ver datos nulos
df_specs_only[df_specs_only['specs'].isnull()] #Los datos nulos, seran considerados como "no data".

,item_id,app_name,specs
88336,777910.0,Robotpencil Presents: Understanding 3D for Con...,None
89048,48220.0,Might & Magic: Heroes VI,None
89183,206760.0,Painkiller: Recurring Evil,None
90103,257610.0,LogoMaker 4,None
90138,262650.0,RaySupreme 3D,None
...,...,...,...
120297,775660.0,Robotpencil Presents: Start with Color,None
120408,755310.0,CyberLink YouCam 7 Deluxe,None
120428,775640.0,Robotpencil Presents: Exercise: Brushwork,None
120429,777930.0,Robotpencil Presents: Creative Composition,None


In [439]:
 #Se reemplazan los datos nulos por "no data"
df_specs_only['specs'].fillna('No data', inplace=True)

In [440]:
def separate_specs(df):
    #Obtiene la lista completa de specs presentes en el DataFrame
    all_specs = set() # Crea un conjunto para almacenar los specs
    for specs in df['specs']:
        if isinstance(specs, list):
            all_specs.update(specs)
            
    # Crea columnas para cada spec encontrado
    for spec in all_specs:
        df[spec] = df['specs'].apply(lambda x: 1 if isinstance(x, list) and spec in x else 0)
        
    #Elimina la columna 'specs' original
    df = df.drop(columns=['specs'])
    
    return df
    

In [441]:
data_specs = separate_specs(df_specs_only)

DATA_SPECS

In [442]:
data_specs.head()

,item_id,app_name,Game demo,Cross-Platform Multiplayer,Co-op,Online Multi-Player,Multi-player,Steam Cloud,Steam Achievements,Steam Trading Cards,...,Local Multi-Player,SteamVR Collectibles,Mods (require HL1),Seated,Shared/Split Screen,Includes Source SDK,HTC Vive,Room-Scale,Single-player,Partial Controller Support
88310,761140.0,Lost Summoner Kitty,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88311,643980.0,Ironbound,0,1,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,1,0
88312,670290.0,Real Pool 3D - Poolians,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88313,767400.0,弹炸人2222,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88314,773570.0,Log Challenge,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0


In [443]:
data_specs.describe()

,item_id,Game demo,Cross-Platform Multiplayer,Co-op,Online Multi-Player,Multi-player,Steam Cloud,Steam Achievements,Steam Trading Cards,Includes level editor,...,Local Multi-Player,SteamVR Collectibles,Mods (require HL1),Seated,Shared/Split Screen,Includes Source SDK,HTC Vive,Room-Scale,Single-player,Partial Controller Support
count,3.213300e+04,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,...,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000,32133.000000
mean,4.517574e+05,0.011297,0.079358,0.133196,0.073756,0.255127,0.309931,0.509290,0.379610,0.042853,...,0.038496,0.000934,0.000031,0.033922,0.123300,0.001867,0.061743,0.044969,0.858183,0.203467
std,1.827140e+05,0.105686,0.270300,0.339792,0.261377,0.435939,0.462472,0.499921,0.485297,0.202529,...,0.192394,0.030541,0.005579,0.181030,0.328786,0.043172,0.240693,0.207240,0.348868,0.402583
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.292800e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,4.520600e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,5.934000e+05,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,2.028850e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [444]:
#Eliminamos las columnas de app name de los dataframes, ya que solo utilizaremos item id para relacionarlos.
data_genre.drop(columns=['app_name'],inplace=True)
data_specs.drop(columns=['app_name'],inplace=True)


In [445]:
data_genre.head()

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
88310,761140.0,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
88311,643980.0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
88312,670290.0,0,0,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
88313,767400.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
88314,773570.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [446]:
data_specs.head()

,item_id,Game demo,Cross-Platform Multiplayer,Co-op,Online Multi-Player,Multi-player,Steam Cloud,Steam Achievements,Steam Trading Cards,Includes level editor,...,Local Multi-Player,SteamVR Collectibles,Mods (require HL1),Seated,Shared/Split Screen,Includes Source SDK,HTC Vive,Room-Scale,Single-player,Partial Controller Support
88310,761140.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88311,643980.0,0,1,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
88312,670290.0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88313,767400.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
88314,773570.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0


In [447]:
#Se eliminan las columnas 'genres' y 'specs' del dataframe original, ya que los datos fueron almacenados en los dataframes 'data_genre' y 'data_specs'.
df_steam_output.drop(columns=['genres', 'specs'],inplace=True)

In [448]:
#Se reorganizan las columnas del dataframe.
df_steam_output = df_steam_output[['item_id', 'app_name', 'developer', 'publisher', 'release_date', 'price', 'early_access', 'url', 'reviews_url']]

In [449]:
df_steam_output.head()

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
88310,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018-01-04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
88311,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018-01-04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017-07-24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
88313,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017-12-07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
88314,773570.0,Log Challenge,None,None,None,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


# FEATURE ENGINEERING

Modificaciones: Se aplica Sentiment Analisys a la columna 'reviews' 

In [450]:
data_users_revs.head()


,user_id,item_id,rev_year,rev_month,rev_day,review,recommend
0,76561197970982479,1250,2011,November,5,Simple yet with great replayability. In my opi...,True
1,76561197970982479,22200,2011,July,15,It's unique and worth a playthrough.,True
2,76561197970982479,43110,2011,April,21,Great atmosphere. The gunplay can be a bit chu...,True
3,js41637,251610,2014,June,24,I know what you think when you see this title ...,True
4,js41637,227300,2013,September,8,For a simple (it's actually not all that simpl...,True


In [451]:
def get_sentiment_label(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    
    if polarity < -0.2:
        return 0  # Malo
    elif polarity > 0.2:
        return 2  # Positivo
    else:
        return 1  # Neutral


In [452]:
# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
data_users_revs['sentiment_analysis'] = data_users_revs['review'].apply(get_sentiment_label)

In [453]:
# Reemplazar la columna 'review' con la nueva columna 'sentiment_analysis'
data_users_revs.drop(columns=['review'], inplace=True)

In [454]:
# Mostrar las primeras filas para verificar los cambios
data_users_revs.head()

,user_id,item_id,rev_year,rev_month,rev_day,recommend,sentiment_analysis
0,76561197970982479,1250,2011,November,5,True,1
1,76561197970982479,22200,2011,July,15,True,2
2,76561197970982479,43110,2011,April,21,True,1
3,js41637,251610,2014,June,24,True,1
4,js41637,227300,2013,September,8,True,1


# NORMALIZACION

Tipos de datos: Se normalizan los datos a utilizar.

## DATA_USERS_ITEMS

In [455]:
#La columna ITEMS_COUNT debe ser transformada a un tipo de dato numerico.
data_users_items['items_count'] = pd.to_numeric(data_users_items['items_count'])
#La nueva columna ITEM_ID debe contener datos de tipo numerico.
data_users_items['item_id'] = pd.to_numeric(data_users_items['item_id'])
#La nueva columna ITEM_NAME debe contener datos de tipo object.
data_users_items['item_name'] = data_users_items['item_name'].astype(str)
#la nueva columna PLAYTIME_FOREVER debe contener datos de tipo numerico.
data_users_items['playtime_forever'] = pd.to_numeric(data_users_items['playtime_forever'])
#la nueva columna PLAYTIME_2WEEKS debe contener datos de tipo numerico.
data_users_items['playtime_2weeks'] = pd.to_numeric(data_users_items['playtime_2weeks'])

In [456]:
data_users_items.head()

,user_id,items_count,steam_id,user_index_count,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,1,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,1,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,1,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,1,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,1,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


## DATA_USERS_REVS

In [457]:
#Se modifica el tipo de dato de la columna ITEM_ID a numerico.
data_users_revs['item_id'] = pd.to_numeric(data_users_revs['item_id'])
#Se unifican las columnas de rev_year, rev_month y rev_day en una sola columna.
data_users_revs['rev_date'] = data_users_revs['rev_year'] + '-' + data_users_revs['rev_month'] + '-' + data_users_revs['rev_day']
#Se modifica el tipo de dato de la columna REV_DATE a fecha.
data_users_revs['rev_date'] = pd.to_datetime(data_users_revs['rev_date'])
#Se elimina la columna REV_YEAR ya que no es necesaria.
data_users_revs.drop(columns=['rev_year'],inplace=True)
#Se elimina la columna REV_MONTH ya que no es necesaria.
data_users_revs.drop(columns=['rev_month'],inplace=True)
#Se elimina la columna REV_DAY ya que no es necesaria.
data_users_revs.drop(columns=['rev_day'],inplace=True)

In [458]:
data_users_revs.head()


,user_id,item_id,recommend,sentiment_analysis,rev_date
0,76561197970982479,1250,True,1,2011-11-05
1,76561197970982479,22200,True,2,2011-07-15
2,76561197970982479,43110,True,1,2011-04-21
3,js41637,251610,True,1,2014-06-24
4,js41637,227300,True,1,2013-09-08


## DATA_STEAM_OUTPUT

In [459]:
data_steam_output.head()

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
88310,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018.0-01-04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
88311,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018.0-01-04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017.0-07-24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
88313,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017.0-12-07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
88314,773570.0,Log Challenge,None,None,nan-None-None,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


In [460]:
#DATA_STEAM_OUTPUT
#Se elimina el ".0" entre de los valores de la columna 'release_date'
data_steam_output['release_date'] = data_steam_output['release_date'].str.replace('.0', '')
#Se transforma la columna 'release_date' que no sea fecha en NaN
data_steam_output['release_date'] = pd.to_datetime(data_steam_output['release_date'], errors='coerce')
#Se modifica el tipo de dato de la columna ITEM_ID a numerico.
data_steam_output['item_id'] = pd.to_numeric(data_steam_output['item_id'])
#Se reemplaza el valor 'Free' de la columna PRICE por 0.00.
data_steam_output['price'] = data_steam_output['price'].replace('Free', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free Demo', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Play for Free!', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Install Now', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Play WARMACHINE: Tactics Demo', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free Mod', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Install Theme', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free Movie', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free HITMAN™ Holiday Pack', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Third-party', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Starting at $499.00', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Play Now', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free to Try', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Free to Use', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Play the Demo', '0.00')
data_steam_output['price'] = data_steam_output['price'].replace('Starting at $449.00', '0.00')

#Se modifica el tipo de dato de la columna PRICE a numerico.
data_steam_output['price'] = pd.to_numeric(data_steam_output['price'])
#Se modifica el tipo de dato de la columna EARLY_ACCESS a numerico.
data_steam_output['early_access'] = pd.to_numeric(data_steam_output['early_access'])
#Se modifica el tipo de dato de la columna RELEASE_DATE a fecha.
data_steam_output['release_date'] = pd.to_datetime(data_steam_output['release_date'])

In [461]:
#Valores de la columna relase date
data_steam_output['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
2007-03-04      1
2006-08-18      1
2006-11-03      1
2007-10-17      1
2018-10-01      1
Name: count, Length: 3457, dtype: int64

In [462]:
#Ver datos nulos de price
data_steam_output[data_steam_output['price'].isnull()]


,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
88319,768800.0,Race,RewindApp,RewindApp,2018-01-04,NaN,0.0,http://store.steampowered.com/app/768800/Race/,http://steamcommunity.com/app/768800/reviews/?...
88320,768570.0,Uncanny Islands,Qucheza,Qucheza,NaT,NaN,1.0,http://store.steampowered.com/app/768570/Uncan...,http://steamcommunity.com/app/768570/reviews/?...
88341,520680.0,Lost Cities,BlueLine Games,BlueLine Games,2018-01-01,NaN,0.0,http://store.steampowered.com/app/520680/Lost_...,http://steamcommunity.com/app/520680/reviews/?...
88342,690410.0,Twisted Enhanced Edition,Games by Brundle,Games by Brundle,2018-01-01,NaN,0.0,http://store.steampowered.com/app/690410/Twist...,http://steamcommunity.com/app/690410/reviews/?...
88344,413120.0,Tactics Forever,ProjectorGames,ProjectorGames,2018-01-01,NaN,0.0,http://store.steampowered.com/app/413120/Tacti...,http://steamcommunity.com/app/413120/reviews/?...
...,...,...,...,...,...,...,...,...,...
120409,771070.0,Infinos Gaiden,Picorinne Soft,Picorinne Soft,2018-01-19,NaN,0.0,http://store.steampowered.com/app/771070/Infin...,http://steamcommunity.com/app/771070/reviews/?...
120421,90007.0,International Online Soccer,I.O.S. Team,None,2002-01-01,NaN,0.0,http://store.steampowered.com/app/90007/Intern...,http://steamcommunity.com/app/90007/reviews/?b...
120433,772180.0,Cricket Club,VersoVR,VersoVR,NaT,NaN,1.0,http://store.steampowered.com/app/772180/Crick...,http://steamcommunity.com/app/772180/reviews/?...
120435,771810.0,The spy who shot me™,Retro Army Limited,Retro Army Limited,2018-10-01,NaN,0.0,http://store.steampowered.com/app/771810/The_s...,http://steamcommunity.com/app/771810/reviews/?...


In [463]:
#se buscan datos duplicados en la columna 'item_id'.
data_steam_output['item_id'].duplicated().sum() 

1

In [464]:
#Ver datos duplicados en la columna 'item_id'.
data_steam_output[data_steam_output['item_id'].duplicated()]

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
102883,612880.0,Wolfenstein II: The New Colossus,Machine Games,Bethesda Softworks,2017-10-26,59.99,0.0,http://store.steampowered.com/app/612880/Wolfe...,http://steamcommunity.com/app/612880/reviews/?...


In [465]:
#ver el item id 612880
data_steam_output[data_steam_output['item_id'] == 612880]

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
102204,612880.0,Wolfenstein II: The New Colossus,Machine Games,Bethesda Softworks,2017-10-26,59.99,0.0,http://store.steampowered.com/app/612880/,http://steamcommunity.com/app/612880/reviews/?...
102883,612880.0,Wolfenstein II: The New Colossus,Machine Games,Bethesda Softworks,2017-10-26,59.99,0.0,http://store.steampowered.com/app/612880/Wolfe...,http://steamcommunity.com/app/612880/reviews/?...


In [466]:
#Se elimina el dato duplicado en la columna 'item_id', que posee un error en la carga de la URL = http://store.steampowered.com/app/612880/
data_steam_output = data_steam_output.drop_duplicates(subset='item_id', keep='first')

In [467]:
data_steam_output.head()

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
88310,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018-01-04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
88311,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018-01-04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
88312,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017-07-24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
88313,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017-12-07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
88314,773570.0,Log Challenge,None,None,NaT,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


# DATABASES PARA API

In [468]:
data_steam_output.to_csv('Datasets/data_steam_output.csv', index=False) #Peso 2.5+ MB
data_users_items.to_csv('Datasets/data_users_items.csv', index=False) #Peso 353.8+ MB
data_users_revs.to_csv('Datasets/data_users_revs.csv', index=False) #Peso 1.9+ GB
data_genre.to_csv('Datasets/data_genre.csv', index=False) #Peso 5.9 MB
data_specs.to_csv('Datasets/data_specs.csv', index=False) #Peso 10.3 MB